In [38]:
import os
import pymongo
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../../config/.env')
mongo_uri = os.getenv('MONGO_URI')
client = pymongo.MongoClient(f'{mongo_uri}')
db = client['paper_data']
collection = db['clean_cu_data']

df = pd.DataFrame(list(collection.find({}, {'_id': 0})))
collection = db['clean_scopus_engineer']
df = pd.concat([df, pd.DataFrame(list(collection.find({}, {'_id': 0})))])

In [39]:
display(df)

,name,affiliation,subject_code,cited_count,pub_type,year
0,Krit,Thailand,2700,1.0,Book,2018
1,Krit,United States,2700,1.0,Book,2018
2,Teerapong,Thailand,2208,1.0,Conference Proceeding,2018
3,Panuwat,Thailand,2208,1.0,Conference Proceeding,2018
4,Teerapong,Thailand,2504,1.0,Conference Proceeding,2018
...,...,...,...,...,...,...
9979,Noelia,Unknown,2305,24.0,Journal,2017
9980,Adrian,Unknown,2305,24.0,Journal,2017
9981,Jonathan,Unknown,2305,24.0,Journal,2017
9982,Ailette,Unknown,2305,24.0,Journal,2017


In [28]:
import pickle
def features(name):
    name = str(name).lower()
    return {
        'first-letter': name[0],
        'last-letter': name[-1],
        'first2-letter': name[:2],
        'last2-letter': name[-2:],
        'first3-letter': name[:3],
        'last3-letter': name[-3:],
        'first4-letter': name[:4],
        'last4-letter': name[-4:]
    }
with open('../model/name_pipeline.pkl', 'rb') as f:
    name_pipeline = pickle.load(f)

In [40]:
prediction = name_pipeline.predict(df['name'])
df['predict_gender'] = prediction

In [42]:
df['cited_count'] = pd.to_numeric(df['cited_count'], errors='coerce').fillna(0).astype(int)
df.to_csv('data_predicted.csv', index=False)

